In [1]:
import numpy as np
from tabulate import tabulate
import pandas as pd
import csv
import ocel as ol
filename = "running-example"

In [2]:
# ocel_test = ol.import_log('newdata/%s.jsonocel' % filename)

In [3]:
def unique_values(df, df_column):
    table = [["column name", "number of unique values", "unique_values"]]
    for col in df_column:
        table.append([str(col), str(len(df.loc[:, col].unique())), str(df.loc[:, col].unique())])
    print(tabulate(table, headers="firstrow",  tablefmt="fancy_grid"))

In [4]:
ocel = pd.read_csv('newdata/%s.csv' % filename)
ocel['ocel:timestamp']=  pd.to_datetime(ocel['ocel:timestamp'])
col_transform = {"ocel:eid": "Event_ID", "ocel:timestamp": "Timestamp", "ocel:activity": "Activity", "ocel:type:items": "Items",
                 "ocel:type:products": "Products", "ocel:type:customers":"Customers", "ocel:type:orders":"Orders",
                 "ocel:type:packages":"Packages"}
ocel = ocel.rename(columns=col_transform)
coltoinspect = ocel.columns
unique_values(ocel,coltoinspect)
ocel['Items']=  ocel['Items'].apply(lambda x: x.strip('][').split(', '))
ocel['Items']=  ocel['Items'].apply(lambda x: [t.replace("'","") for t in x])
ocel['Products']=  ocel['Products'].apply(lambda x: x.strip('][').split(', '))
ocel['Products']=  ocel['Products'].apply(lambda x: [t.replace("'","") for t in x])
ocel['Customers']=  ocel['Customers'].apply(lambda x: x.strip('][').split(', '))
ocel['Customers']=  ocel['Customers'].apply(lambda x: [t.replace("'","") for t in x])
ocel['Orders']=  ocel['Orders'].apply(lambda x: x.strip('][').split(', '))
ocel['Orders']=  ocel['Orders'].apply(lambda x: [t.replace("'","") for t in x])
ocel['Packages']=  ocel['Packages'].fillna('')
ocel['Packages']=  ocel['Packages'].apply(lambda x: x.strip('][').split(', '))
ocel['Packages']=  ocel['Packages'].apply(lambda x: [t.replace("'","") for t in x])
ocel['Packages']=  ocel['Packages'].apply(lambda x: x[-1])
ocel['Customers']=  ocel['Customers'].apply(lambda x: x[-1])
ocel.head()

╒═══════════════╤═══════════════════════════╤═════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╕
│ column name   │   number of unique values │ unique_values                                                                                                                                                                           │
╞═══════════════╪═══════════════════════════╪═════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╡
│ Event_ID      │                     22367 │ [1.0000e+00 2.0000e+00 3.0000e+00 ... 2.2365e+04 2.2366e+04 2.2367e+04]                                                                                                                 │
├───────────────┼───────────────────────────┼───────────────────────────

,Event_ID,Timestamp,Activity,weight,price,Items,Products,Customers,Orders,Packages
0,1.0,2019-05-20 09:07:47,place order,3.520,524.96,"[880001, 880004, 880003, 880002]","[Echo Studio, Echo Show 8, Fire Stick 4K, Echo]",Marco Pegoraro,[990001],
1,2.0,2019-05-20 10:35:21,place order,2.656,3255.99,"[880008, 880005, 880006, 880007]","[Kindle, iPad Air, iPad, MacBook Air]",Gyunam Park,[990002],
2,3.0,2019-05-20 10:38:17,pick item,0.483,79.99,[880006],[Kindle],Gyunam Park,[990002],
3,4.0,2019-05-20 11:13:54,confirm order,3.520,524.96,"[880001, 880004, 880003, 880002]","[Echo Studio, Echo Show 8, Fire Stick 4K, Echo]",Marco Pegoraro,[990001],
4,5.0,2019-05-20 11:20:13,pick item,0.280,89.99,[880002],[Fire Stick 4K],Marco Pegoraro,[990001],


In [5]:
col = ["Items","Products","Customers","Orders","Packages"]
for c in col:
    print(c)
    print(np.unique([len(t) for t in ocel[c]]))

Items
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22]
Products
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14]
Customers
[11 12 13 14 16 17 20 22 27]
Orders
[1 2 3 4 5 6 7 8 9]
Packages
[0 6]


In [6]:
t1 = [len(t) for t in ocel['Items']]
t2 = [len(t) for t in ocel['Products']]
t3 = [a!=b for a, b in zip(t1, t2)]
print(ocel.size)
print(sum(t3))
print(sum(t3)/ocel.size)
display(ocel[t3])
ocel[t3].explode('Products').groupby(['Products']).size().sort_values()

223670
4174
0.01866142084320651


,Event_ID,Timestamp,Activity,weight,price,Items,Products,Customers,Orders,Packages
19,20.0,2019-05-21 07:13:12,place order,3.013,1370.98,"[880032, 880033, 880031, 880034, 880030, 880029]","[Kindle Paperwhite, iPad mini, Kindle, Echo Sh...",Gyunam Park,[990009],
37,38.0,2019-05-21 13:40:12,confirm order,3.013,1370.98,"[880032, 880033, 880031, 880034, 880030, 880029]","[Kindle Paperwhite, iPad mini, Kindle, Echo Sh...",Gyunam Park,[990009],
39,40.0,2019-05-21 13:52:38,place order,3.237,2761.99,"[880046, 880047, 880045, 880044, 880043, 88004...","[iPad Air, iPhone X, Kindle Paperwhite, Echo S...",Gyunam Park,[990012],
49,50.0,2019-05-21 16:05:03,place order,3.194,3127.97,"[880049, 880053, 880048, 880050, 880051, 880052]","[iPhone 11 Pro, Echo Studio, iPad Pro, iPad, F...",Gyunam Park,[990013],
50,51.0,2019-05-21 16:44:36,confirm order,3.237,2761.99,"[880046, 880047, 880045, 880044, 880043, 88004...","[iPad Air, iPhone X, Kindle Paperwhite, Echo S...",Gyunam Park,[990012],
...,...,...,...,...,...,...,...,...,...,...
22324,22325.0,2020-06-25 15:21:47,payment reminder,2.714,2171.98,"[887172, 887170, 887171, 887168, 887169]","[iPhone X, Echo Show 5, iPhone 8, Echo Plus]",Gyunam Park,[991750],
22325,22326.0,2020-06-28 01:00:00,create package,2.438,4601.00,"[888142, 888143, 887801, 888141]","[iPhone 11 Pro, iPad Air, MacBook Pro]",Junxiong Gao,"[991911, 991994]",661314
22326,22327.0,2020-06-29 13:28:26,send package,2.438,4601.00,"[888142, 888143, 887801, 888141]","[iPhone 11 Pro, iPad Air, MacBook Pro]",Junxiong Gao,"[991911, 991994]",661314
22328,22329.0,2020-06-29 15:25:50,package delivered,2.438,4601.00,"[888142, 888143, 887801, 888141]","[iPhone 11 Pro, iPad Air, MacBook Pro]",Junxiong Gao,"[991911, 991994]",661314


Products
Kindle                934
MacBook Air          1029
MacBook Pro          1077
Echo Show 5          1082
iPad mini            1092
iPad                 1113
Echo Dot             1117
Kindle Paperwhite    1122
iPhone 11 Pro        1123
iPad Air             1134
Echo Show 8          1136
Echo Studio          1148
Echo Plus            1154
Fire Stick           1160
Fire Stick 4K        1170
iPhone 11            1183
iPhone X             1190
iPad Pro             1200
iPhone 8             1240
Echo                 1247
dtype: int64

In [7]:
df_agg = ocel.copy()
df_agg.Items = df_agg.Items.apply(len)
df_agg.Orders = df_agg.Orders.apply(len)
df_agg.Products = df_agg.Products.apply(len)
df_agg.Customers = df_agg.Customers.apply(len)
df_agg = df_agg.groupby('Packages').sum()
df_agg

C:\Users\kwein\AppData\Local\Temp\ipykernel_1456\1220846720.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_agg = df_agg.groupby('Packages').sum()


,Event_ID,weight,price,Items,Products,Customers,Orders
Packages,,,,,,,
,200453766.0,23193.639,21642829.56,38114,35519,261587,18001
660001,567.0,28.844,32679.80,64,44,44,20
660002,527.0,12.819,11066.91,15,15,42,6
660003,1017.0,29.604,12595.80,48,32,48,16
660004,701.0,14.265,13979.88,33,27,36,12
...,...,...,...,...,...,...,...
661321,67062.0,0.840,269.97,3,3,33,3
661322,67071.0,1.485,387.00,3,3,48,3
661323,67081.0,1.449,239.97,3,3,39,3


In [18]:
print(ocel.size)
display(ocel[ocel['Event_ID']== 594])
# ocel = ocel.explode('Items')
ocel_orders = ocel.explode('Orders').reset_index(drop=True)
print(ocel_orders.size)
display(ocel_orders[ocel_orders['Event_ID']== 594])
ocel_orders.head()

223670


,Event_ID,Timestamp,Activity,weight,price,Items,Products,Customers,Orders,Packages
593,594.0,2019-05-31 16:18:19,package delivered,3.613,5193.97,"[880186, 880187, 880066, 880233, 880188, 88022...","[iPhone X, iPad, iPhone 11 Pro, Fire Stick, iP...",Christina Rensinghof,"[990016, 990056, 990061, 990060, 990048]",660020


324470


,Event_ID,Timestamp,Activity,weight,price,Items,Products,Customers,Orders,Packages
742,594.0,2019-05-31 16:18:19,package delivered,3.613,5193.97,"[880186, 880187, 880066, 880233, 880188, 88022...","[iPhone X, iPad, iPhone 11 Pro, Fire Stick, iP...",Christina Rensinghof,990016,660020
743,594.0,2019-05-31 16:18:19,package delivered,3.613,5193.97,"[880186, 880187, 880066, 880233, 880188, 88022...","[iPhone X, iPad, iPhone 11 Pro, Fire Stick, iP...",Christina Rensinghof,990056,660020
744,594.0,2019-05-31 16:18:19,package delivered,3.613,5193.97,"[880186, 880187, 880066, 880233, 880188, 88022...","[iPhone X, iPad, iPhone 11 Pro, Fire Stick, iP...",Christina Rensinghof,990061,660020
745,594.0,2019-05-31 16:18:19,package delivered,3.613,5193.97,"[880186, 880187, 880066, 880233, 880188, 88022...","[iPhone X, iPad, iPhone 11 Pro, Fire Stick, iP...",Christina Rensinghof,990060,660020
746,594.0,2019-05-31 16:18:19,package delivered,3.613,5193.97,"[880186, 880187, 880066, 880233, 880188, 88022...","[iPhone X, iPad, iPhone 11 Pro, Fire Stick, iP...",Christina Rensinghof,990048,660020


,Event_ID,Timestamp,Activity,weight,price,Items,Products,Customers,Orders,Packages
0,1.0,2019-05-20 09:07:47,place order,3.520,524.96,"[880001, 880004, 880003, 880002]","[Echo Studio, Echo Show 8, Fire Stick 4K, Echo]",Marco Pegoraro,990001,
1,2.0,2019-05-20 10:35:21,place order,2.656,3255.99,"[880008, 880005, 880006, 880007]","[Kindle, iPad Air, iPad, MacBook Air]",Gyunam Park,990002,
2,3.0,2019-05-20 10:38:17,pick item,0.483,79.99,[880006],[Kindle],Gyunam Park,990002,
3,4.0,2019-05-20 11:13:54,confirm order,3.520,524.96,"[880001, 880004, 880003, 880002]","[Echo Studio, Echo Show 8, Fire Stick 4K, Echo]",Marco Pegoraro,990001,
4,5.0,2019-05-20 11:20:13,pick item,0.280,89.99,[880002],[Fire Stick 4K],Marco Pegoraro,990001,


In [17]:
ocel_orders.size

324470

In [29]:
ocel_orders.set_index('Orders').drop(columns=["weight","price","Customers","Event_ID"])[120:140]

,Timestamp,Activity,Items,Products,Packages
Orders,,,,,
990011,2019-05-23 11:00:25,create package,"[880032, 880038, 880006, 880051, 880040, 88000...","[iPad Air, iPhone X, iPad, iPhone 8, Kindle, E...",660001
990002,2019-05-23 11:00:25,create package,"[880032, 880038, 880006, 880051, 880040, 88000...","[iPad Air, iPhone X, iPad, iPhone 8, Kindle, E...",660001
990013,2019-05-23 11:00:25,create package,"[880032, 880038, 880006, 880051, 880040, 88000...","[iPad Air, iPhone X, iPad, iPhone 8, Kindle, E...",660001
990012,2019-05-23 11:00:25,create package,"[880032, 880038, 880006, 880051, 880040, 88000...","[iPad Air, iPhone X, iPad, iPhone 8, Kindle, E...",660001
990009,2019-05-23 11:00:25,create package,"[880032, 880038, 880006, 880051, 880040, 88000...","[iPad Air, iPhone X, iPad, iPhone 8, Kindle, E...",660001
990022,2019-05-23 11:07:19,item out of stock,[880091],[iPad mini],
990015,2019-05-23 11:08:24,item out of stock,[880062],[iPad Air],
990022,2019-05-23 11:23:05,confirm order,"[880090, 880089, 880091, 880088]","[iPhone X, iPad mini, Echo Plus, MacBook Pro]",
990001,2019-05-23 11:27:55,pay order,"[880001, 880004, 880003, 880002]","[Echo Studio, Echo Show 8, Fire Stick 4K, Echo]",


In [30]:
ocel_orders.set_index('Orders').drop(columns=["weight","price","Items","Products","Customers","Packages","Event_ID"])[120:140]

,Timestamp,Activity
Orders,,
990011,2019-05-23 11:00:25,create package
990002,2019-05-23 11:00:25,create package
990013,2019-05-23 11:00:25,create package
990012,2019-05-23 11:00:25,create package
990009,2019-05-23 11:00:25,create package
990022,2019-05-23 11:07:19,item out of stock
990015,2019-05-23 11:08:24,item out of stock
990022,2019-05-23 11:23:05,confirm order
990001,2019-05-23 11:27:55,pay order


In [42]:
t1 = [len(t) for t in ocel['Items']]
t2 = [len(t) for t in ocel['Products']]
t3 = [a==b for a, b in zip(t1, t2)]
oceltemp = ocel[t3].copy()
t1 = [len(t) for t in oceltemp['Items']]
t2 = [len(t) for t in oceltemp['Products']]
t3 = [a!=b for a, b in zip(t1, t2)]
sum(t3)

0

In [44]:
list('AC')
print(ocel.size)
display(ocel[ocel['Event_ID']== 594])
ocel_items = oceltemp.explode(['Items','Products']).reset_index(drop=True)
print(ocel_items.size)
display(ocel_items[ocel_items['Event_ID']== 594])
ocel_items.head()

223670


,Event_ID,Timestamp,Activity,weight,price,Items,Products,Customers,Orders,Packages
593,594.0,2019-05-31 16:18:19,package delivered,3.613,5193.97,"[880186, 880187, 880066, 880233, 880188, 88022...","[iPhone X, iPad, iPhone 11 Pro, Fire Stick, iP...",Christina Rensinghof,"[990016, 990056, 990061, 990060, 990048]",660020


355070


,Event_ID,Timestamp,Activity,weight,price,Items,Products,Customers,Orders,Packages


,Event_ID,Timestamp,Activity,weight,price,Items,Products,Customers,Orders,Packages
0,1.0,2019-05-20 09:07:47,place order,3.520,524.96,880001,Echo Studio,Marco Pegoraro,[990001],
1,1.0,2019-05-20 09:07:47,place order,3.520,524.96,880004,Echo Show 8,Marco Pegoraro,[990001],
2,1.0,2019-05-20 09:07:47,place order,3.520,524.96,880003,Fire Stick 4K,Marco Pegoraro,[990001],
3,1.0,2019-05-20 09:07:47,place order,3.520,524.96,880002,Echo,Marco Pegoraro,[990001],
4,2.0,2019-05-20 10:35:21,place order,2.656,3255.99,880008,Kindle,Gyunam Park,[990002],


In [124]:
ocel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22367 entries, 0 to 22366
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Event_ID   22367 non-null  float64       
 1   Timestamp  22367 non-null  datetime64[ns]
 2   Activity   22367 non-null  object        
 3   weight     22367 non-null  float64       
 4   price      22367 non-null  float64       
 5   Items      22367 non-null  object        
 6   Products   22367 non-null  object        
 7   Customers  22367 non-null  object        
 8   Orders     22367 non-null  object        
 9   Packages   22367 non-null  object        
dtypes: datetime64[ns](1), float64(3), object(6)
memory usage: 1.7+ MB
